<a href="https://colab.research.google.com/github/alejandra74/Topicos_de_Informacao_I/blob/main/prova01_alejandrainga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prova P1 - FT043 (Fundamentos da Ciência de Dados) – 2S2022**

O treinamento de um classificador supervisionado é realizado por um conjunto de  dados de entrada X e o respectivo vetor de rótulos d, no qual cada elemento indica a classe associada a uma certa amostra. Considerando o conjunto de dados fornecidos, que contém atributos discretos e um vetor de rótulos, realize as seguintes etapas.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
dados = pd.read_excel("https://github.com/alejandra74/Topicos_de_Informacao_I/blob/90eed90cca58bab82000a8b6f8262ccfcf0a3f28/dados_P1.xlsx?raw=true")

In [3]:
dados.head()

,x1,x2,d
0,1.0,1.0,1.0
1,0.0,0.0,-1.0
2,1.0,1.0,-1.0
3,2.0,2.0,1.0
4,3.0,3.0,1.0


In [4]:
dados.describe()

,x1,x2,d
count,1500.000000,1500.000000,1500.000000
mean,1.446667,1.239333,-0.333333
std,0.880723,0.915029,0.943123
min,0.000000,0.000000,-1.000000
25%,1.000000,1.000000,-1.000000
50%,1.000000,1.000000,-1.000000
75%,2.000000,2.000000,1.000000
max,3.000000,3.000000,1.000000


In [5]:
# Quantidade de dados com rotulo -1
print("Quantidade de dados com rotulo -1:",(dados[dados.d == -1]).shape[0])
# Quantidade de dados com rotulo 1
print("Quantidade de dados com rotulo 1:",(dados[dados.d == 1]).shape[0])

Quantidade de dados com rotulo -1: 1000
Quantidade de dados com rotulo 1: 500


In [6]:
X = dados.iloc[:,:-1]
X.head()

,x1,x2
0,1.0,1.0
1,0.0,0.0
2,1.0,1.0
3,2.0,2.0
4,3.0,3.0


In [7]:
y = dados.iloc[:,-1]

### (a) (2 pontos) Defina um classificador considerando a distribuição de probabilidade dos atributos para cada uma das classes. Forneça a regra de classificação associada a tal classificador.

In [8]:
Px1 = dados.pivot_table('x1', index = 'x1', columns='d', aggfunc='count',fill_value = 0)/[1000,500]
Px1

d,-1.0,1.0
x1,,
0.0,0.218,0.000
1.0,0.432,0.280
2.0,0.350,0.364
3.0,0.000,0.356


In [9]:
Px2 = dados.pivot_table('x2', index = 'x2', columns='d', aggfunc='count',fill_value = 0)/[1000,500]
Px2

d,-1.0,1.0
x2,,
0.0,0.365,0.000
1.0,0.383,0.308
2.0,0.252,0.440
3.0,0.000,0.252


In [10]:
Px1_1 = np.array(Px1[-1])
Px1_2 = np.array(Px1[1])
Px2_1 = np.array(Px2[-1])
Px2_2 = np.array(Px2[1])

In [11]:
#P(X=-1)
p1 = (dados[dados.d == -1]).shape[0] / dados.shape[0]
p1

0.6666666666666666

In [12]:
#P(X=1)
p2 = (dados[dados.d == 1]).shape[0] / dados.shape[0]
p2

0.3333333333333333

In [13]:
def ClaseDados(x,Px1_1,Px1_2,Px2_1,Px2_2):
  x1 = x[0]
  x2 = x[1]

  def CalcMatris(x):
    if x == 0:
      matris = [1,0,0,0]
    elif x == 1:
      matris = [0,1,0,0]
    elif x == 2:
      matris = [0,0,1,0]
    else:
      matris = [0,0,0,1]
    return matris

  matris1 = CalcMatris(x1)
  matris2 = CalcMatris(x2)

  d1_1 = matris1*Px1_1
  d1_2 = matris1*Px1_2
  d2_1 = matris2*Px2_1
  d2_2 = matris2*Px2_2

  P1 = [d1_1, d2_1]
  P2 = [d1_2, d2_2]

  for i in range(len(P1)):
    P1[i] = sum(P1[i])
    P2[i] = sum(P2[i])
  
  Prob_C1 = P1[0]*P1[1]*p1
  Prob_C2 = P2[0]*P2[1]*p2
  
  if Prob_C1>Prob_C2:
    Clase = -1
  else:
    Clase = 1
  
  return Clase

In [14]:
XA = np.array(X)
clase = []
for i in XA:
  clase.append(ClaseDados(i,Px1_1,Px1_2,Px2_1,Px2_2))

### (b) (2 pontos) Calcule a acurácia (proporção de classificações corretas) do classificador obtido em (a).

In [15]:
y = np.array(y)
def acuracia(clase,y):
  k = 0
  for i in range(len(clase)):
    if y[i] != clase[i]:
      k += 1
  return print('Acuracia do classificador é:', round(1-(k/1500),3))

In [16]:
acuracia(clase,y)

Acuracia do classificador é: 0.817


### (c) (2 pontos) Defina um classificador considerando a distribuição de probabilidade dos atributos para cada uma das classes, bem como a proporção das classes. Forneça a regra de classificação associada a tal classificador.

In [49]:
from scipy.stats import norm

In [50]:
def fit_distribution(data):
	# estimate parameters
	mu = np.mean(data)
	sigma = np.std(data)
	print(mu, sigma)
	# fit distribution
	dist = norm(mu, sigma)
	return dist

In [31]:
Xy1 = np.array(X[y == -1])
Xy2 = np.array(X[y == 1])

In [23]:
print(Xy1.shape, Xy2.shape)

(1000, 2) (500, 2)


In [24]:
# calculate priors
priory1 = len(Xy1) / len(X)
priory2 = len(Xy2) / len(X)
print(priory1, priory2)

0.6666666666666666 0.3333333333333333


In [53]:
# create PDFs for y==-1
distX1y1 = fit_distribution(Xy1[:, 0])
distX2y1 = fit_distribution(Xy1[:, 1])
# create PDFs for y==1
distX1y2 = fit_distribution(Xy2[:, 0])
distX2y2 = fit_distribution(Xy2[:, 1])

1.132 0.7420080862093081
0.887 0.7773229701996461
2.076 0.7938664875153756
1.944 0.7462332075162561


In [52]:
# calculate the independent conditional probability
def probability(X, prior, dist1, dist2):
	return prior * dist1.pdf(X[0]) * dist2.pdf(X[1])

In [66]:
Prob1 = []
Prob2 = []
clase_2=[]
for i in XA:
  py1 = probability(i, priory1, distX1y1, distX2y1)
  py2 = probability(i, priory2, distX1y2, distX2y2)
  if py1 > py2:
    c = -1
  else:
    c = 1
  clase_2.append(c)
  Prob1.append(py1)
  Prob2.append(py2)

### (d) (2 pontos) Calcule a acurácia (proporção de classificações corretas) do classificador obtido em (c).

In [67]:
acuracia(clase_2,y)

Acuracia do classificador é: 0.806


### (e) (2 pontos) Compare os resultados obtidos pelos classificadores (a) e (c).